### Rule: find the reordered items in the second to last purchase
### These will be the prediction for the last purchase

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

In [2]:
order_products_train_df = pd.read_csv("input/order_products__train.csv")
order_products_prior_df = pd.read_csv("input/order_products__prior.csv")
orders_df = pd.read_csv("input/orders.csv")
products_df = pd.read_csv("input/products.csv")
aisles_df = pd.read_csv("input/aisles.csv")
departments_df = pd.read_csv("input/departments.csv")

### find the second to last order number per user_id

In [3]:
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
# the last_order_number is between 4~100
orders_df.groupby(["user_id"]).agg(last_order_number=('order_number', 'max')).sort_values("last_order_number")

,last_order_number
user_id,
127633,4
74147,4
160570,4
160560,4
74171,4
...,...
1868,100
31118,100
122476,100


In [5]:
last_orders_number_df = orders_df.groupby(["user_id"]).agg(last_order_number=('order_number', 'max')).reset_index()

In [6]:
# calculate second to last number
last_orders_number_df['second_last_order_number'] = last_orders_number_df['last_order_number'] - 1 

In [7]:
last_orders_number_df.sort_values('second_last_order_number')

,user_id,last_order_number,second_last_order_number
127632,127633,4,3
74146,74147,4,3
160569,160570,4,3
160559,160560,4,3
74170,74171,4,3
...,...,...,...
1867,1868,100,99
31117,31118,100,99
122475,122476,100,99
120896,120897,100,99


In [8]:
# through merge, filter orders_df to contain only second_last_orders
second_last_orders_df = orders_df.merge(last_orders_number_df, left_on=['user_id','order_number'], 
                right_on=['user_id','second_last_order_number'], how='inner')

In [9]:
second_last_orders_df

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order_number,second_last_order_number
0,2550362,1,prior,10,4,8,30.0,11,10
1,839880,2,prior,14,3,10,13.0,15,14
2,1402502,3,prior,12,1,15,15.0,13,12
3,2557754,4,prior,5,5,13,0.0,6,5
4,157374,5,prior,4,1,18,19.0,5,4
...,...,...,...,...,...,...,...,...,...
206204,414137,206205,prior,3,5,16,10.0,4,3
206205,1904200,206206,prior,67,0,13,11.0,68,67
206206,1005822,206207,prior,16,2,7,18.0,17,16
206207,1882108,206208,prior,49,1,22,7.0,50,49


In [10]:
order_products_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [11]:
order_products_prior_df["order_id"].nunique()

3214874

32,434,488 products purchased with 3,214,874 orders

In [12]:
order_products_prior_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


### find the products reordered in the second to last orders

In [13]:
second_last_order_products_prior_df = order_products_prior_df.merge(second_last_orders_df, on='order_id', how='inner')

In [14]:
second_last_order_products_prior_df["order_id"].nunique()

206209

In [15]:
second_last_order_products_prior_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2139788 entries, 0 to 2139787
Data columns (total 12 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   order_id                  int64  
 1   product_id                int64  
 2   add_to_cart_order         int64  
 3   reordered                 int64  
 4   user_id                   int64  
 5   eval_set                  object 
 6   order_number              int64  
 7   order_dow                 int64  
 8   order_hour_of_day         int64  
 9   days_since_prior_order    float64
 10  last_order_number         int64  
 11  second_last_order_number  int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 212.2+ MB


32,434,488 products purchased with 3,214,874 orders \
reduces to 2,139,787 products purchased with 206,209 orders

In [16]:
second_last_order_products_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order_number,second_last_order_number
0,25,9755,1,1,59897,prior,19,6,10,25.0,20,19
1,25,31487,2,0,59897,prior,19,6,10,25.0,20,19
2,25,37510,3,1,59897,prior,19,6,10,25.0,20,19
3,25,14576,4,1,59897,prior,19,6,10,25.0,20,19
4,25,22105,5,0,59897,prior,19,6,10,25.0,20,19


In [17]:
# generate the predictions
predict = second_last_order_products_prior_df[second_last_order_products_prior_df['reordered']==1]

In [18]:
order_products_train_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [19]:
order_products_train_df = order_products_train_df.merge(orders_df, on ="order_id", how ="left")

In [20]:
# about 60 % in the training are reordered
order_products_train_df[order_products_train_df['reordered']==1].shape[0] / order_products_train_df.shape[0]

0.5985944127509629

In [21]:
predict.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254593 entries, 0 to 2139781
Data columns (total 12 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   order_id                  1254593 non-null  int64  
 1   product_id                1254593 non-null  int64  
 2   add_to_cart_order         1254593 non-null  int64  
 3   reordered                 1254593 non-null  int64  
 4   user_id                   1254593 non-null  int64  
 5   eval_set                  1254593 non-null  object 
 6   order_number              1254593 non-null  int64  
 7   order_dow                 1254593 non-null  int64  
 8   order_hour_of_day         1254593 non-null  int64  
 9   days_since_prior_order    1254593 non-null  float64
 10  last_order_number         1254593 non-null  int64  
 11  second_last_order_number  1254593 non-null  int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 124.4+ MB


In [22]:
predict.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order_number,second_last_order_number
0,25,9755,1,1,59897,prior,19,6,10,25.0,20,19
2,25,37510,3,1,59897,prior,19,6,10,25.0,20,19
3,25,14576,4,1,59897,prior,19,6,10,25.0,20,19
5,25,1934,6,1,59897,prior,19,6,10,25.0,20,19
6,25,6383,7,1,59897,prior,19,6,10,25.0,20,19


In [23]:
order_products_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 0 to 1384616
Data columns (total 10 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   order_id                1384617 non-null  int64  
 1   product_id              1384617 non-null  int64  
 2   add_to_cart_order       1384617 non-null  int64  
 3   reordered               1384617 non-null  int64  
 4   user_id                 1384617 non-null  int64  
 5   eval_set                1384617 non-null  object 
 6   order_number            1384617 non-null  int64  
 7   order_dow               1384617 non-null  int64  
 8   order_hour_of_day       1384617 non-null  int64  
 9   days_since_prior_order  1384617 non-null  float64
dtypes: float64(1), int64(8), object(1)
memory usage: 116.2+ MB


In [24]:
order_products_train_df['user_id'].nunique()

131209

In [25]:
predict['user_id'].nunique()

191624

In [26]:
predict = predict[["user_id","product_id","second_last_order_number"]]

In [27]:
# filter user_id to the ones in the train df
predict = predict[predict['user_id'].isin(order_products_train_df['user_id'])]

In [28]:
predict['user_id'].nunique()

121861

We have a lot more people in the predict data set than in the train data set before. After the filter, the unique user_ids is smaller than train. \
Note: for those not predicted, we predict NONE in theory. This edge case will be handled later

In [29]:
result = order_products_train_df.merge(predict, on=["user_id","product_id"], how="outer")

In [30]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1878367 entries, 0 to 1878366
Data columns (total 11 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   order_id                  float64
 1   product_id                int64  
 2   add_to_cart_order         float64
 3   reordered                 float64
 4   user_id                   int64  
 5   eval_set                  object 
 6   order_number              float64
 7   order_dow                 float64
 8   order_hour_of_day         float64
 9   days_since_prior_order    float64
 10  second_last_order_number  float64
dtypes: float64(8), int64(2), object(1)
memory usage: 172.0+ MB


In [31]:
result.sample(5)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,second_last_order_number
218009,533840.0,25931,7.0,1.0,129736,train,5.0,0.0,11.0,23.0,4.0
1454661,NaN,46979,NaN,NaN,125688,NaN,NaN,NaN,NaN,NaN,13.0
95691,237369.0,42342,6.0,0.0,29728,train,4.0,6.0,14.0,2.0,NaN
952127,2348624.0,6878,2.0,1.0,21086,train,13.0,4.0,20.0,9.0,12.0
610917,1503135.0,31066,4.0,1.0,1209,train,9.0,6.0,19.0,18.0,NaN


### calculate true positive, true negative, false positive, false positive
precision = true positive/ (true positive + false positive) \
recall = true positive/ (true positive + false negative) \
F1 score = 2/ (1/precision + 1/recall)

In [32]:
result.shape[0]

1878367

In [33]:
result.loc[result["order_id"].notnull() & result["second_last_order_number"].notnull(), "tp"] = 1

In [34]:
result.loc[result["order_id"].isnull() & result["second_last_order_number"].isnull(), "tn"] = 1

In [35]:
result.loc[result["order_id"].isnull() & result["second_last_order_number"].notnull(), "fp"] = 1

In [36]:
result.loc[result["order_id"].notnull() & result["second_last_order_number"].isnull(), "fn"] = 1

In [37]:
result['tp'] = result['tp'].fillna(0)
result['tn'] = result['tn'].fillna(0)
result['fp'] = result['fp'].fillna(0)
result['fn'] = result['fn'].fillna(0)

In [38]:
# this is to correct the cases that we don't have none in our data
result.loc[(result["tn"]==0) & (result["tp"]==0) & (result["fp"]==0), "fp"] = 1

In [39]:
result_user = result.groupby(["user_id"])["tp","tn","fp","fn"].sum().reset_index()

/var/folders/32/_lk9s_9n7pggt5bls7z52jk00000gn/T/ipykernel_90477/1976579316.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result_user = result.groupby(["user_id"])["tp","tn","fp","fn"].sum().reset_index()


In [40]:
result_user.head()

,user_id,tp,tn,fp,fn
0,1,5.0,0.0,7.0,6.0
1,2,2.0,0.0,37.0,29.0
2,5,2.0,0.0,13.0,7.0
3,7,3.0,0.0,15.0,6.0
4,8,1.0,0.0,21.0,17.0


In [41]:
result_user['precision'] = result_user["tp"] / (result_user["tp"] + result_user["fp"])

In [42]:
result_user['recall'] = result_user["tp"] / (result_user["tp"] + result_user["fn"])

In [43]:
result_user['f1'] = 2/ (1/result_user['precision'] + 1/result_user['recall'])

In [44]:
result_user.sort_values("precision")

,user_id,tp,tn,fp,fn,precision,recall,f1
65604,103150,0.0,0.0,5.0,5.0,0.0,0.0,0.0
57176,89841,0.0,0.0,6.0,6.0,0.0,0.0,0.0
57175,89840,0.0,0.0,11.0,10.0,0.0,0.0,0.0
57174,89837,0.0,0.0,4.0,1.0,0.0,0.0,0.0
57171,89833,0.0,0.0,17.0,16.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
41406,65223,1.0,0.0,0.0,0.0,1.0,1.0,1.0
12607,19716,1.0,0.0,0.0,0.0,1.0,1.0,1.0
36297,57135,2.0,0.0,0.0,0.0,1.0,1.0,1.0
30237,47562,4.0,0.0,0.0,0.0,1.0,1.0,1.0


In [45]:
result_user['f1'].mean()

0.18844655416352427

For our simple rule of using second to last orders for predictions, our F1 score is 18.84